# Advanced LangGraph Patterns

## Introduction

This notebook covers advanced agent patterns:
- **ReAct**: Reasoning and Acting
- **Plan-and-Execute**: Strategic planning
- **Reflection**: Self-correction
- **Multi-Agent**: Collaborative systems
- **Hierarchical**: Task decomposition

These patterns are essential for building sophisticated AI agents.

## Pattern 1: ReAct (Reason + Act)

### Concept

ReAct combines reasoning and acting in an interleaved manner:

```
Thought: What do I need to do?
Action: [Take action]
Observation: [See result]
Thought: What does this mean?
Action: [Take another action]
...
Thought: I now know the answer
Final Answer: [Response]
```

### Implementation

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Literal
import operator

class ReActState(TypedDict):
    input: str
    thoughts: Annotated[list[str], operator.add]
    actions: Annotated[list[dict], operator.add]
    observations: Annotated[list[str], operator.add]
    final_answer: str | None
    iteration: int

# Simulated tools
def search_tool(query: str) -> str:
    return f"Search results for '{query}': Information about {query}..."

def calculate_tool(expr: str) -> str:
    try:
        return str(eval(expr))
    except:
        return "Error in calculation"

TOOLS = {
    "search": search_tool,
    "calculate": calculate_tool
}

def reason(state: ReActState) -> ReActState:
    """Agent thinks about what to do next"""
    # In practice, use LLM to generate thought
    if state["iteration"] == 0:
        thought = f"I need to search for information about: {state['input']}"
    elif state["iteration"] < 3:
        thought = f"Based on observation, I should continue investigating"
    else:
        thought = "I have enough information to answer"
    
    return {"thoughts": [thought], "iteration": state["iteration"]}

def act(state: ReActState) -> ReActState:
    """Agent takes an action"""
    # In practice, LLM decides which tool to use
    last_thought = state["thoughts"][-1]
    
    if "search" in last_thought.lower():
        action = {"tool": "search", "input": state["input"]}
    elif "calculate" in last_thought.lower():
        action = {"tool": "calculate", "input": "2+2"}
    else:
        action = None
    
    if action:
        # Execute tool
        result = TOOLS[action["tool"]](action["input"])
        return {
            "actions": [action],
            "observations": [result],
            "iteration": state["iteration"]
        }
    else:
        return {"iteration": state["iteration"]}

def decide_next(state: ReActState) -> Literal["reason", "finish"]:
    """Decide whether to continue reasoning or finish"""
    if state["iteration"] >= 3 or "enough information" in state["thoughts"][-1]:
        return "finish"
    return "reason"

def finalize(state: ReActState) -> ReActState:
    """Generate final answer"""
    final_answer = f"Based on {len(state['observations'])} observations, here's my answer..."
    return {"final_answer": final_answer, "iteration": state["iteration"]}

# Build ReAct graph
react_workflow = StateGraph(ReActState)

react_workflow.add_node("reason", reason)
react_workflow.add_node("act", act)
react_workflow.add_node("finalize", finalize)

react_workflow.set_entry_point("reason")
react_workflow.add_edge("reason", "act")
react_workflow.add_conditional_edges(
    "act",
    decide_next,
    {"reason": "reason", "finish": "finalize"}
)
react_workflow.add_edge("finalize", END)

react_agent = react_workflow.compile()

# Test
result = react_agent.invoke({
    "input": "quantum computing",
    "thoughts": [],
    "actions": [],
    "observations": [],
    "final_answer": None,
    "iteration": 0
})

print("ReAct Pattern Example")
print("=" * 50)
print(f"\nThoughts: {result['thoughts']}")
print(f"\nActions taken: {len(result['actions'])}")
print(f"\nFinal answer: {result['final_answer']}")

## Pattern 2: Plan-and-Execute

### Concept

Separate planning from execution:

1. **Planner**: Creates a step-by-step plan
2. **Executor**: Executes each step
3. **Re-planner**: Adjusts plan based on results

```
┌────────┐
│  Plan  │ → [Step 1, Step 2, Step 3]
└────┬───┘
     │
     ▼
┌─────────┐
│ Execute │ → Do Step 1
└────┬────┘
     │
     ▼
┌─────────┐
│Re-plan? │ → Adjust if needed
└────┬────┘
     │
     ▼
   Repeat
```

In [ ]:
class PlanExecuteState(TypedDict):
    input: str
    plan: list[str]
    past_steps: Annotated[list[tuple[str, str]], operator.add]  # (step, result)
    current_step_index: int
    final_response: str | None

def create_plan(state: PlanExecuteState) -> PlanExecuteState:
    """Generate initial plan"""
    # In practice, use LLM to generate plan
    task = state["input"]
    
    plan = [
        f"Search for information about {task}",
        "Analyze the information",
        "Synthesize findings",
        "Format final answer"
    ]
    
    return {"plan": plan, "current_step_index": 0}

def execute_step(state: PlanExecuteState) -> PlanExecuteState:
    """Execute current step of the plan"""
    current_index = state["current_step_index"]
    
    if current_index >= len(state["plan"]):
        return state
    
    current_step = state["plan"][current_index]
    
    # Execute the step (simplified)
    result = f"Completed: {current_step}"
    
    return {
        "past_steps": [(current_step, result)],
        "current_step_index": current_index + 1
    }

def replan(state: PlanExecuteState) -> PlanExecuteState:
    """Adjust plan based on results"""
    # In practice, LLM reviews past steps and adjusts plan
    # For now, keep plan unchanged
    return state

def should_continue_plan(state: PlanExecuteState) -> Literal["execute", "finish"]:
    """Check if plan is complete"""
    if state["current_step_index"] >= len(state["plan"]):
        return "finish"
    return "execute"

def finalize_plan(state: PlanExecuteState) -> PlanExecuteState:
    """Create final response"""
    summary = f"Completed {len(state['past_steps'])} steps"
    return {"final_response": summary}

# Build Plan-Execute graph
plan_exec_workflow = StateGraph(PlanExecuteState)

plan_exec_workflow.add_node("planner", create_plan)
plan_exec_workflow.add_node("executor", execute_step)
plan_exec_workflow.add_node("replanner", replan)
plan_exec_workflow.add_node("finalizer", finalize_plan)

plan_exec_workflow.set_entry_point("planner")
plan_exec_workflow.add_edge("planner", "executor")
plan_exec_workflow.add_edge("executor", "replanner")
plan_exec_workflow.add_conditional_edges(
    "replanner",
    should_continue_plan,
    {"execute": "executor", "finish": "finalizer"}
)
plan_exec_workflow.add_edge("finalizer", END)

plan_exec_agent = plan_exec_workflow.compile()

# Test
result = plan_exec_agent.invoke({
    "input": "Analyze AI agent frameworks",
    "plan": [],
    "past_steps": [],
    "current_step_index": 0,
    "final_response": None
})

print("Plan-Execute Pattern Example")
print("=" * 50)
print(f"\nPlan: {result['plan']}")
print(f"\nSteps executed: {len(result['past_steps'])}")
print(f"\nFinal response: {result['final_response']}")

## Pattern 3: Reflection

### Concept

Agent reviews and improves its own output:

```
┌─────────┐
│Generate │ → Draft answer
└────┬────┘
     │
     ▼
┌─────────┐
│ Reflect │ → Find issues
└────┬────┘
     │
  [Issues?]
     │
  Yes ────► Revise ──┐
     │               │
    No               │
     │               │
     ▼               │
  Finalize ◄─────────┘
```

In [ ]:
class ReflectionState(TypedDict):
    task: str
    draft: str
    critiques: Annotated[list[str], operator.add]
    revision_count: int
    final_output: str | None

def generate_draft(state: ReflectionState) -> ReflectionState:
    """Create initial draft"""
    # In practice, use LLM to generate
    draft = f"Draft response for: {state['task']}"
    return {"draft": draft, "revision_count": 0}

def reflect_on_draft(state: ReflectionState) -> ReflectionState:
    """Critique the current draft"""
    # In practice, use LLM to critique
    critiques = [
        "Could be more specific",
        "Needs more examples"
    ]
    
    # Only add critiques on first few iterations
    if state["revision_count"] < 2:
        return {"critiques": critiques, "revision_count": state["revision_count"]}
    return {"revision_count": state["revision_count"]}

def revise_draft(state: ReflectionState) -> ReflectionState:
    """Improve draft based on critiques"""
    # In practice, use LLM to revise
    revised = state["draft"] + f" (Revision {state['revision_count'] + 1})"
    return {
        "draft": revised,
        "revision_count": state["revision_count"] + 1
    }

def should_revise(state: ReflectionState) -> Literal["revise", "accept"]:
    """Decide if we need another revision"""
    # Revise if we have critiques and haven't exceeded max revisions
    has_critiques = len(state["critiques"]) > state["revision_count"]
    under_limit = state["revision_count"] < 3
    
    if has_critiques and under_limit:
        return "revise"
    return "accept"

def accept_draft(state: ReflectionState) -> ReflectionState:
    """Accept current draft as final"""
    return {"final_output": state["draft"]}

# Build Reflection graph
reflection_workflow = StateGraph(ReflectionState)

reflection_workflow.add_node("generate", generate_draft)
reflection_workflow.add_node("reflect", reflect_on_draft)
reflection_workflow.add_node("revise", revise_draft)
reflection_workflow.add_node("accept", accept_draft)

reflection_workflow.set_entry_point("generate")
reflection_workflow.add_edge("generate", "reflect")
reflection_workflow.add_conditional_edges(
    "reflect",
    should_revise,
    {"revise": "revise", "accept": "accept"}
)
reflection_workflow.add_edge("revise", "reflect")
reflection_workflow.add_edge("accept", END)

reflection_agent = reflection_workflow.compile()

# Test
result = reflection_agent.invoke({
    "task": "Explain LangGraph",
    "draft": "",
    "critiques": [],
    "revision_count": 0,
    "final_output": None
})

print("Reflection Pattern Example")
print("=" * 50)
print(f"\nRevisions made: {result['revision_count']}")
print(f"\nCritiques addressed: {len(result['critiques'])}")
print(f"\nFinal output: {result['final_output']}")

## Pattern 4: Multi-Agent Collaboration

### Concept

Multiple specialized agents working together:

```
        ┌───────────┐
        │Supervisor │
        └─────┬─────┘
              │
      ┌───────┼───────┐
      │       │       │
      ▼       ▼       ▼
  ┌────┐  ┌────┐  ┌────┐
  │ A1 │  │ A2 │  │ A3 │
  └────┘  └────┘  └────┘
 Researcher Coder Reviewer
```

In [ ]:
class MultiAgentState(TypedDict):
    task: str
    messages: Annotated[list[dict], operator.add]
    next_agent: str | None
    final_result: str | None

def supervisor(state: MultiAgentState) -> MultiAgentState:
    """Decide which agent should act next"""
    # In practice, use LLM to route
    task = state["task"]
    
    if "research" in task.lower():
        next_agent = "researcher"
    elif "code" in task.lower():
        next_agent = "coder"
    elif "review" in task.lower():
        next_agent = "reviewer"
    else:
        next_agent = "FINISH"
    
    return {
        "next_agent": next_agent,
        "messages": [{"from": "supervisor", "to": next_agent, "content": task}]
    }

def researcher_agent(state: MultiAgentState) -> MultiAgentState:
    """Research specialist"""
    result = "Research findings: ..."
    return {
        "messages": [{"from": "researcher", "content": result}],
        "next_agent": "supervisor"
    }

def coder_agent(state: MultiAgentState) -> MultiAgentState:
    """Coding specialist"""
    code = "def solution(): pass"
    return {
        "messages": [{"from": "coder", "content": code}],
        "next_agent": "supervisor"
    }

def reviewer_agent(state: MultiAgentState) -> MultiAgentState:
    """Review specialist"""
    review = "Review: Looks good!"
    return {
        "messages": [{"from": "reviewer", "content": review}],
        "next_agent": "supervisor"
    }

def route_to_agent(state: MultiAgentState) -> str:
    """Route to next agent or finish"""
    return state["next_agent"] or "FINISH"

# Build multi-agent graph
multi_agent_workflow = StateGraph(MultiAgentState)

multi_agent_workflow.add_node("supervisor", supervisor)
multi_agent_workflow.add_node("researcher", researcher_agent)
multi_agent_workflow.add_node("coder", coder_agent)
multi_agent_workflow.add_node("reviewer", reviewer_agent)

multi_agent_workflow.set_entry_point("supervisor")

# Conditional edges from supervisor
multi_agent_workflow.add_conditional_edges(
    "supervisor",
    route_to_agent,
    {
        "researcher": "researcher",
        "coder": "coder",
        "reviewer": "reviewer",
        "FINISH": END
    }
)

# All agents return to supervisor
multi_agent_workflow.add_edge("researcher", "supervisor")
multi_agent_workflow.add_edge("coder", "supervisor")
multi_agent_workflow.add_edge("reviewer", "supervisor")

multi_agent_system = multi_agent_workflow.compile()

# Test
result = multi_agent_system.invoke({
    "task": "Research and code a solution",
    "messages": [],
    "next_agent": None,
    "final_result": None
})

print("Multi-Agent Pattern Example")
print("=" * 50)
print(f"\nMessages exchanged: {len(result['messages'])}")
for msg in result['messages']:
    print(f"  {msg}")

## Pattern 5: Hierarchical Agents

### Concept

Complex tasks broken into subtasks with specialized sub-agents:

```
┌──────────────┐
│Master Agent  │
│Task: Build   │
│  full app    │
└──────┬───────┘
       │
   ┌───┴────┬────────┐
   ▼        ▼        ▼
┌─────┐  ┌─────┐  ┌─────┐
│Sub1 │  │Sub2 │  │Sub3 │
│  Backend   │   Frontend  │  Tests
└─────┘  └─────┘  └─────┘
```

## Production Patterns

### 1. Circuit Breaker

In [ ]:
class CircuitBreakerState(TypedDict):
    attempts: int
    max_attempts: int
    errors: Annotated[list[str], operator.add]
    circuit_open: bool

def try_operation(state: CircuitBreakerState) -> CircuitBreakerState:
    if state["circuit_open"]:
        return {"errors": ["Circuit breaker is open"]}
    
    # Simulate operation that might fail
    import random
    if random.random() < 0.5:  # 50% failure rate
        return {
            "attempts": state["attempts"] + 1,
            "errors": [f"Attempt {state['attempts'] + 1} failed"]
        }
    else:
        return {"attempts": state["attempts"] + 1}

def check_circuit(state: CircuitBreakerState) -> CircuitBreakerState:
    """Open circuit if too many errors"""
    if len(state["errors"]) >= 3:
        return {"circuit_open": True}
    return state

### 2. Fallback Strategy

In [ ]:
def try_primary_model(state: dict) -> dict:
    try:
        # Try expensive, accurate model
        result = expensive_llm.invoke(state["messages"])
        state["result"] = result
        state["model_used"] = "primary"
    except Exception:
        state["use_fallback"] = True
    return state

def try_fallback_model(state: dict) -> dict:
    # Use cheaper, faster model
    result = cheap_llm.invoke(state["messages"])
    state["result"] = result
    state["model_used"] = "fallback"
    return state

## Best Practices

### 1. Choose the Right Pattern

| Use Case | Recommended Pattern |
|----------|--------------------|
| Simple Q&A with tools | ReAct |
| Complex projects | Plan-and-Execute |
| Quality critical | Reflection |
| Specialized tasks | Multi-Agent |
| Large tasks | Hierarchical |

### 2. Hybrid Approaches

Combine patterns for better results:
- Plan-and-Execute + Reflection
- Multi-Agent + ReAct
- Hierarchical + Plan-and-Execute

### 3. Error Handling

In [ ]:
def safe_node_wrapper(node_func):
    def wrapper(state):
        try:
            return node_func(state)
        except Exception as e:
            return {
                "error": str(e),
                "failed_node": node_func.__name__
            }
    return wrapper

## Key Takeaways

- **ReAct**: Best for interactive, tool-using agents
- **Plan-Execute**: Ideal for complex, multi-step tasks
- **Reflection**: Improves output quality through self-critique
- **Multi-Agent**: Leverages specialization for complex problems
- **Hierarchical**: Handles large-scale task decomposition
- **Production patterns**: Circuit breakers, fallbacks, error handling

## Next Steps

- Build real-world applications
- Optimize for cost and latency
- Add monitoring and observability
- Deploy to production

---

**Next**: [05_practical_examples.ipynb](./05_practical_examples.ipynb)